初始状态下，点击所有位置Reward暂定为0

In [ ]:
from MineSweeper.core import MineSweeperService

iter = 1000
mineSweeperService = MineSweeperService(height=6,width = 6,mineNum = 6)
for _ in range(iter):
    mineSweeperService.RandomClick()
    curGame = mineSweeperService.GetCurGame()


TypeError: 'module' object is not callable